Get the system up-to-date

In [ ]:
!sudo apt-get update
!sudo apt-get -y upgrade
!sudo apt -y autoremove
!pip install --upgrade pip

Install system dependencies

In [ ]:
!sudo apt-get -y install libgdal-dev

Install python packages

In [ ]:
!pip install python-dotenv gdal==3.0.4

Load env vars file

In [ ]:
%load_ext dotenv
%dotenv local.env

In [ ]:
import os
import io
import requests
from dotenv import load_dotenv
from osgeo import gdal

load_dotenv("local.env")
username = os.getenv("EARTHDATA_USERNAME")
password = os.getenv("EARTHDATA_PASSWORD")
tiles_url_root = os.getenv("TILES_URL_ROOT") # The url prefix to use for the tile file names
tiles_file_path = os.getenv("TILES_FILE_PATH") # The path to a file containing a list of file names
tiles_file = open(tiles_file_path) # The file containing a list of file names
tile_file_names = tiles_file.readlines() # The list of file names

gdal.SetConfigOption('GDAL_PAM_ENABLED', 'NO')

def process_tile_file(tile_file_path):
    raster = gdal.Open(tile_file_path, gdal.GA_Update)
    band = raster.GetRasterBand(1)
    band.SetNoDataValue(-9999)
    raster.FlushCache()
    del band
    del raster

def download_tiles(tiles_url_root, tile_file_names, earthdata_username, earthdata_password, output_path):
    with requests.Session() as session:
        session.auth = (username, password)
        
        for i, tile_file_name in enumerate(tile_file_names):
    
            progress = f'[{i+1}/{len(tile_file_names)}]'
            destination_path = os.path.join(output_path, tile_file_name.strip())
            download_url = os.path.join(tiles_url_root, tile_file_name.strip())
    
            if (os.path.isfile(destination_path)):
                print(f'{progress} File {destination_path} already exists')
                continue;
            
            print(f"{progress} Downloading '{download_url}'")
            flightRequest = session.request('get', download_url) # Make a flight request to get a session cookie
            contentRequest = session.get(flightRequest.url, auth=(earthdata_username, earthdata_password))
            if contentRequest.ok:
                file = open(destination_path, 'wb')
                file.write(contentRequest.content)
                file.close()
                process_tile_file(destination_path)
            else:
                print("Error: " + contentRequest.status_code + ", " + contentRequest.text)
        print('Done!')

output_path = "earthdata_tiles"
os.makedirs(output_path, exist_ok=True)
download_tiles(
    tiles_url_root, tile_file_names,
    username, password,
    output_path
)
